## Linear Programming

A solution for the Decentral assessment. The problem that is being solved is "A Political Problem" in Chapter 29 of CLRS

### SciPy implementation

In [7]:
from scipy.optimize import linprog

In [8]:
obj = [1, 1, 1, 1]
lhs_ineq = [[2, -8, 0, -10],
            [-5, -2, 0, 0],
            [-3, 5, -10, 2]]
rhs_ineq = [-50,
            -100,
            -25]
bnd = [(0, float("inf")),
       (0, float("inf")),
       (0, float("inf")),
       (0, float("inf"))]

opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
              bounds=bnd,
              method="simplex")
# methods can also be 'interior-point', 'revised simplex', 'simplex'

In [9]:
print(f'Values of x1, x2, x3, x4 are respectively: ')
print(opt.x[:])

print(f'\nUrban vote count is')
print(f"-2({opt.x[0]}) + 8({opt.x[1]}) + 0({opt.x[2]}) + 10({opt.x[3]}) = {(-2*(opt.x[0]) + 8*(opt.x[1]) + 10*(opt.x[3]))}")

print(f'\nSubUrban vote count is')
print(f"5({opt.x[0]}) + 2({opt.x[1]}) + 0({opt.x[2]}) + 0({opt.x[3]}) = {(5*(opt.x[0]) + 2*(opt.x[1]))}")

print(f'\nRural vote count is')
print(f"3({opt.x[0]}) - 5({opt.x[1]}) + 10({opt.x[2]}) - 2({opt.x[3]}) = {(3*(opt.x[0]) - 5*(opt.x[1]) + 10*(opt.x[2]) - 2*(opt.x[3]))}")

print(f'\nAnd total amount spent in thousands is')
print(sum(opt.x))
# opt.x

Values of x1, x2, x3, x4 are respectively: 
[18.46846847  3.82882883  0.          5.63063063]

Urban vote count is
-2(18.46846846846847) + 8(3.8288288288288266) + 0(0.0) + 10(5.630630630630631) = 49.999999999999986

SubUrban vote count is
5(18.46846846846847) + 2(3.8288288288288266) + 0(0.0) + 0(5.630630630630631) = 100.0

Rural vote count is
3(18.46846846846847) - 5(3.8288288288288266) + 10(0.0) - 2(5.630630630630631) = 25.000000000000007

And total amount spent in thousands is
27.927927927927925


### PuLP implementation

In [10]:
from pulp import LpMinimize, LpProblem, LpStatus, LpVariable

In [11]:
model = LpProblem(name="small-problem", sense=LpMinimize)

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)
x3 = LpVariable(name="x3", lowBound=0)
x4 = LpVariable(name="x4", lowBound=0)

model += (-2*x1 + 8*x2 + 0*x3 + 10*x4 >= 50, "urban_constraint")
model += (5*x1 + 2*x2 + 0*x3 + 0*x4 >= 100, "suburban_constraint")
model += (3*x1 - 5*x2 + 10*x3 - 2*x4 >= 25, "rural_constraint")

obj_func = x1 + x2 + x3 + x4
model += obj_func

status = model.solve()

In [13]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")

status: 1, Optimal
objective: 27.9279274
x1: 18.468468
x2: 3.8288288
x3: 0.0
x4: 5.6306306


In [14]:
model = LpProblem(name="small-problem", sense=LpMinimize)

x1 = LpVariable(name="x1", lowBound=0, cat="Integer")
x2 = LpVariable(name="x2", lowBound=0, cat="Integer")
x3 = LpVariable(name="x3", lowBound=0, cat="Integer")
x4 = LpVariable(name="x4", lowBound=0, cat="Integer")

model += (-2*x1 + 8*x2 + 0*x3 + 10*x4 >= 50, "urban_constraint")
model += (5*x1 + 2*x2 + 0*x3 + 0*x4 >= 100, "suburban_constraint")
model += (3*x1 - 5*x2 + 10*x3 - 2*x4 >= 25, "rural_constraint")

obj_func = x1 + x2 + x3 + x4
model += obj_func

status = model.solve()

In [15]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")

status: 1, Optimal
objective: 29.0
x1: 18.0
x2: 5.0
x3: 1.0
x4: 5.0


### Coding simplex algorithm from scratch

In [6]:
import numpy as np

# Generate the initial tableau for performing simplex algorithm
def gen_matrix(var,cons):
    tab = np.zeros((cons+1, var+cons+2))
    return tab

# Find if a negative value existing in the last column. Function can output either boolean or index
def find_neg_r(table, bool = False):
    lc = len(table[0,:])
    m = min(table[:-1,lc-1])
    if m<=0:
        n = np.where(table[:-1,lc-1] == m)[0][0]
    else:
        n = None
    if(bool):
        if m>= 0:
            return False
        else:
            return True
    return n

# Find if a negative value existing in the last row. Function can output either boolean or index
def find_neg(table, bool = False):
    lr = len(table[:,0])
    m = min(table[lr-1,:-1])
    if m<=0:
        n = np.where(table[lr-1,:-1] == m)[0][0]
    else:
        n = None
    if(bool):
        if m>= 0:
            return False
        else:
            return True
    return n

# To remove negative values from last column, identify the pivot element and return row, col values of that element
def loc_piv_r(table):
    total = []
    r = find_neg_r(table)
    row = table[r,:-1]
    m = min(row)
    c = np.where(row == m)[0][0]
    col = table[:-1,c]
    for i, b in zip(col,table[:-1,-1]):
        if i**2>0 and b/i>0:
            total.append(b/i)
        else:
            total.append(0)
    element = max(total)
    for t in total:
        if t > 0 and t < element:
            element = t
        else:
            continue

    index = total.index(element)
    return [index,c]

# To remove negative values from last row, identify the pivot element and return row, col values of that element
def loc_piv(table):
    if find_neg(table, bool=True):
        total = []
        n = find_neg(table)
        for i,b in zip(table[:-1,n],table[:-1,-1]):
            if i**2>0 and b/i>0:
                total.append(b/i)
            else:
                total.append(0)
        element = max(total)
        for t in total:
            if t > 0 and t < element:
                element = t
            else:
                continue

        index = total.index(element)
        return [index,n]

# Convert equation into float values for adding to tableau
def convert(eq):
    eq = eq.split(',')
    if 'G' in eq:
        g = eq.index('G')
        del eq[g]
        eq = [float(i)*-1 for i in eq]
        return eq
    if 'L' in eq:
        l = eq.index('L')
        del eq[l]
        eq = [float(i) for i in eq]
        return eq

# In case of a minimization problem, final row is multiplied by -1 making it similar to a maximization problem
def convert_min(table):
    table[-1,:-2] = [-1*i for i in table[-1,:-2]]
    table[-1,-1] = -1*table[-1,-1]
    return table

# Pivot the tableau based on a specific row, col
def pivot(row,col,table):
    lr = len(table[:,0])
    lc = len(table[0,:])
    t = np.zeros((lr,lc))
    pr = table[row,:]
    if table[row,col]**2>0:
        e = 1/table[row,col]
        r = pr*e
        for i in range(len(table[:,col])):
            k = table[i,:]
            c = table[i,col]
            if list(k) == list(pr):
                continue
            else:
                t[i,:] = list(k-r*c)
        t[row,:] = list(r)
        return t
    else:
        print('Cannot pivot on this element.')

# Function to add constraint values to tableau
def constraint(table,eq):
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    j = 0
    flag = 0
    while j < lr:
        row_check = table[j,:]
        total = 0
        for i in row_check:
            total += float(i**2)
        if total == 0:
            flag = 1
            row = row_check
            break
        j +=1
    if(flag==0 or j==lr-1):
        print('Cannot add any more constraints')
        return
    eq = convert(eq)
    i = 0
    while i<len(eq)-1:
        row[i] = eq[i]
        i +=1
    row[-1] = eq[-1]

    row[var+j] = 1

# Function to add objective function values to tableau
def obj(table,eq):
    lr = len(table[:,0])
    j = 0
    flag = 0
    while j < lr:
        row_check = table[j,:]
        total = 0
        for i in row_check:
            total += float(i**2)
        if total == 0:
            flag = 1
            row = row_check
            break
        j +=1
    if(flag == 0 or j!=lr-1):
        print('Cannot add obj function before adding all constraints')
        return
    eq = [float(i) for i in eq.split(',')]
    lr = len(table[:,0])
    row = table[lr-1,:]
    i = 0
    while i<len(eq)-1:
        row[i] = eq[i]*-1
        i +=1
    row[-2] = 1
    row[-1] = eq[-1]

# Function to perform the simplex algorithm after initial tableau has been created
def optimize_prob(table, minimization = True):
    if(minimization):
        table = convert_min(table)
    while find_neg_r(table, bool=True)==True:
        table = pivot(loc_piv_r(table)[0],loc_piv_r(table)[1],table)
    while find_neg(table, bool=True)==True:
        table = pivot(loc_piv(table)[0],loc_piv(table)[1],table)

    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    var_names = []
    for i in range(var):
        var_names.append('x'+str(i+1))
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]
            val[var_names[i]] = table[loc,-1]
        else:
            val[var_names[i]] = 0
    if(minimization):
        val['min'] = table[-1,-1]*-1
    else:
        val['min'] = table[-1,-1]
    for k,v in val.items():
        val[k] = round(v,5)
    return val

if __name__ == "__main__":

    m = gen_matrix(4,3)
    constraint(m, '-2,8,0,10,G,50')
    constraint(m, '5,2,0,0,G,100')
    constraint(m, '3,-5,10,-2,G,25')
    obj(m,'1,1,1,1,0')
    print(optimize_prob(m, minimization=True))

{'x1': 18.46847, 'x2': 3.82883, 'x3': 0, 'x4': 5.63063, 'min': 27.92793}
